**Data Preparation with Pandas**

Importing the Dataset and Initial Exploration

In [70]:
import pandas as pd

lab1_df=pd.read_csv('https://data.cityofnewyork.us/api/views/5rq2-4hqu/rows.csv?accessType=DOWNLOAD')

lab1_df.head()

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_assem,st_senate,nta,nta_name,boro_ct,state,Latitude,longitude,x_sp,y_sp
0,08/27/2015,180683,348711,POINT (-73.84421521958048 40.723091773924274),3,0,OnCurb,Alive,Fair,Acer rubrum,...,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.768749
1,09/03/2015,200540,315986,POINT (-73.81867945834878 40.79411066708779),21,0,OnCurb,Alive,Fair,Quercus palustris,...,27,11,QN49,Whitestone,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.837379
2,09/05/2015,204026,218365,POINT (-73.93660770459083 40.717580740099116),3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,50,18,BK90,East Williamsburg,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.891267
3,09/05/2015,204337,217969,POINT (-73.93445615919741 40.713537494833226),10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,53,18,BK90,East Williamsburg,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.253136
4,08/30/2015,189565,223043,POINT (-73.97597938483258 40.66677775537875),21,0,OnCurb,Alive,Good,Tilia americana,...,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.425999


Creating a Reduced DatFrame

In [71]:
columns_of_interest = ['spc_latin', 'spc_common', 'status', 'health', 'tree_dbh', 'stump_diam']
lab1_df_reduced = lab1_df[columns_of_interest]

lab1_df_reduced

,spc_latin,spc_common,status,health,tree_dbh,stump_diam
0,Acer rubrum,red maple,Alive,Fair,3,0
1,Quercus palustris,pin oak,Alive,Fair,21,0
2,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,3,0
3,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,10,0
4,Tilia americana,American linden,Alive,Good,21,0
...,...,...,...,...,...,...
683783,Quercus palustris,pin oak,Alive,Good,25,0
683784,Cladrastis kentukea,Kentucky yellowwood,Alive,Good,7,0
683785,Acer rubrum,red maple,Alive,Good,12,0
683786,Acer rubrum,red maple,Alive,Good,9,0


In [10]:
print(f"Number of rows in the reduced DataFrame: {len(lab1_df_reduced)}")

Number of rows in the reduced DataFrame: 683788


Handling NaN Values

In [72]:
nullValue_counts = lab1_df_reduced.isnull().sum()
print(nullValue_counts)

spc_latin     31619
spc_common    31619
status            0
health        31616
tree_dbh          0
stump_diam        0
dtype: int64


In [73]:
NaNsOnly = lab1_df_reduced.loc[(lab1_df_reduced['spc_latin'].isnull()) |
                                lab1_df_reduced['spc_common'].isnull() |
                                lab1_df_reduced['health'].isnull()]
NaNsOnly.head()

,spc_latin,spc_common,status,health,tree_dbh,stump_diam
61,NaN,NaN,Dead,NaN,2,0
307,NaN,NaN,Stump,NaN,0,9
370,NaN,NaN,Dead,NaN,4,0
494,NaN,NaN,Stump,NaN,0,17
556,NaN,NaN,Dead,NaN,2,0


In [74]:
pd.value_counts(NaNsOnly['status'])

status
Stump    17654
Dead     13961
Alive        6
Name: count, dtype: int64

Filtering Out Rows with Zero Diameter

In [75]:
zeros_filter = (lab1_df_reduced['tree_dbh'] == 0) & (lab1_df_reduced['stump_diam'] == 0)
zero_diam_rows = lab1_df_reduced.loc[zeros_filter]
print(f"Number of rows where both diameters are zero: {len(zero_diam_rows)}")


Number of rows where both diameters are zero: 278


In [76]:
lab1_df_reduced_zerosFiltered = lab1_df_reduced.loc[~zeros_filter]
print(f"Original DataFrame length: {len(lab1_df_reduced)}")
print(f"Filtered DataFrame length: {len(lab1_df_reduced_zerosFiltered)}")
print(f"Number of rows removed: {len(lab1_df_reduced) - len(lab1_df_reduced_zerosFiltered)}")

Original DataFrame length: 683788
Filtered DataFrame length: 683510
Number of rows removed: 278


Grouping by specified columns and calculate the mean tree diameter

In [80]:
grouped_columns = ['status', 'spc_latin', 'spc_common', 'health']
lab1_df_reduced_zerosFiltered_grouped = lab1_df_reduced_zerosFiltered.groupby(grouped_columns, dropna=False).mean().reset_index()

In [81]:
lab1_df_reduced_zerosFiltered_grouped['diameter'] = lab1_df_reduced_zerosFiltered_grouped['tree_dbh'] + lab1_df_reduced_zerosFiltered_grouped['stump_diam']

lab1_df_reduced_zerosFiltered_grouped.drop(columns=['tree_dbh', 'stump_diam'], inplace=True)

lab1_df_reduced_zerosFiltered_grouped

,status,spc_latin,spc_common,health,diameter
0,Alive,Acer,maple,Fair,11.903991
1,Alive,Acer,maple,Good,11.778865
2,Alive,Acer,maple,Poor,10.509434
3,Alive,Acer buergerianum,trident maple,Fair,9.750000
4,Alive,Acer buergerianum,trident maple,Good,7.724138
...,...,...,...,...,...
393,Alive,NaN,NaN,Good,13.000000
394,Alive,NaN,NaN,Poor,11.000000
395,Dead,Gleditsia triacanthos var. inermis,honeylocust,NaN,12.000000
396,Dead,NaN,NaN,NaN,5.492017


Grouping by Specified Columns and Sum the Counts

In [111]:
grouped_columns = ['status', 'spc_latin', 'spc_common', 'health']
lab1_df_reduced_counts_grouped = lab1_df_reduced_counts.groupby(grouped_columns, dropna=False).sum().reset_index()

lab1_df_reduced_counts_grouped

,status,spc_latin,spc_common,health,tree_dbh,stump_diam,count
0,Alive,Acer,maple,Fair,20582,0,1733
1,Alive,Acer,maple,Good,55396,0,4708
2,Alive,Acer,maple,Poor,6684,0,639
3,Alive,Acer buergerianum,trident maple,Fair,195,0,20
4,Alive,Acer buergerianum,trident maple,Good,672,0,87
...,...,...,...,...,...,...,...
393,Alive,NaN,NaN,Good,52,0,4
394,Alive,NaN,NaN,Poor,11,0,1
395,Dead,Gleditsia triacanthos var. inermis,honeylocust,NaN,12,0,1
396,Dead,NaN,NaN,NaN,76361,0,13960


Merging DataFrames

In [114]:
lab1_df_final = pd.merge(lab1_df_reduced_counts_grouped, lab1_df_reduced_zerosFiltered_grouped,
                         on=grouped_columns, how='left')
lab1_df_final = lab1_df_final.set_index(['status', 'spc_latin', 'spc_common', 'health'])
lab1_df_final

level_0  \
status spc_latin                          spc_common    health            
Alive  Acer                               maple         Fair          0   
                                                        Good          1   
                                                        Poor          2   
       Acer buergerianum                  trident maple Fair          3   
                                                        Good          4   
...                                                                 ...   
       NaN                                NaN           Good        393   
                                                        Poor        394   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN         395   
       NaN                                NaN           NaN         396   
Stump  NaN                                NaN           NaN         397   

                                                                index  \
status spc_latin                          spc_common    health          
Alive  Acer                               maple         Fair        0   
                                                        Good        1   
                                                        Poor        2   
       Acer buergerianum                  trident maple Fair        3   
                                                        Good        4   
...                                                               ...   
       NaN                                NaN           Good      393   
                                                        Poor      394   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN       395   
       NaN                                NaN           NaN       396   
Stump  NaN                                NaN           NaN       397   

                                                                tree_dbh  \
status spc_latin                          spc_common    health             
Alive  Acer                               maple         Fair       20582   
                                                        Good       55396   
                                                        Poor        6684   
       Acer buergerianum                  trident maple Fair         195   
                                                        Good         672   
...                                                                  ...   
       NaN                                NaN           Good          52   
                                                        Poor          11   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN           12   
       NaN                                NaN           NaN        76361   
Stump  NaN                                NaN           NaN            0   

                                                                stump_diam  \
status spc_latin                          spc_common    health               
Alive  Acer                               maple         Fair             0   
                                                        Good             0   
                                                        Poor             0   
       Acer buergerianum                  trident maple Fair             0   
                                                        Good             0   
...                                                                    ...   
       NaN                                NaN           Good             0   
                                                        Poor             0   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN              0   
       NaN                                NaN           NaN              0   
Stump  NaN                                NaN           NaN         295713   

                                                                count  \
status spc_latin                          spc_common    healt

Save the final DataFrame as a CSV file

In [115]:
lab1_df_final.to_csv('lab1_df_final.csv')

print("CSV file has been saved successfully.")

CSV file has been saved successfully.
